In [76]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly as py
from pprint import pprint
import requests
import json
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go

py.offline.init_notebook_mode(connected=True)


#reading raw data
data1 = pd.read_csv("Inpatient_Prospective_Payment_System.csv",encoding='mac_roman')
data2 = pd.read_csv("bedsLatLong.csv",encoding='mac_roman')
data3 = pd.read_csv("ratingsAndOwnership.csv",encoding='mac_roman')
data4 = pd.read_csv("USA costs with latlon.csv",encoding='mac_roman')

In [77]:
#selecting the columns 
data1 = data1[['Provider Id', 'Provider Name',
       'Provider Street Address', 'Provider City', 'Provider State',
       'Provider Zip Code',     
       ' Average Total Payments ', 'Average Medicare Payments']]


data2=data2[[ 'ID','NAME',  'BEDS']]

data2 = data2.rename(columns={'ID': 'Provider Id', 'NAME':'Provider Name'})

data3= data3[['Provider ID', 'Hospital Type', 'Hospital Ownership',
      
       'Hospital overall rating'
      ]]

data3 = data3.rename(columns={'Provider ID': 'Provider Id'})

data4 =data4[['Provider Id','Latitude',
       'Longitude']]
data4['Provider Id'] = pd.to_numeric(data4['Provider Id'])

In [78]:
#Merg the initial 3 data set
main = pd.merge(data1,data3, on = 'Provider Id', how = 'outer')
main = pd.merge(main,data2, on = 'Provider Name', how = 'outer')




In [79]:
#do some clean up
main= main.drop_duplicates()

main = main.dropna(how = "any") 

main = main[['Provider Id_x', 'Provider Name', 'Provider Street Address',
       'Provider City', 'Provider State', 'Provider Zip Code',
       ' Average Total Payments ', 'Average Medicare Payments',
       'Hospital Type', 'Hospital Ownership', 'Hospital overall rating',
        'BEDS']]
main = main.rename(columns={'Provider Id_x': 'Provider Id'})

In [80]:
main.columns

Index(['Provider Id', 'Provider Name', 'Provider Street Address',
       'Provider City', 'Provider State', 'Provider Zip Code',
       ' Average Total Payments ', 'Average Medicare Payments',
       'Hospital Type', 'Hospital Ownership', 'Hospital overall rating',
       'BEDS'],
      dtype='object')

In [81]:
#grouping catagory into Hospital ownership: Voluntary(non-profit), Government, Individual business
main["Hospital Ownership"] = main["Hospital Ownership"].replace({
    'Voluntary non-profit - Private':'Voluntary(non-profit)',
    'Voluntary non-profit - Church':'Voluntary(non-profit)',
    'Voluntary non-profit - Other':'Voluntary(non-profit)', 
    'Government - Hospital District or Authority': 'Government',
    'Government - State': 'Government',
    'Government - Local': 'Government',
    'Government - Federal': 'Government',
    'Proprietary': 'Individual Business',
     'Physician': 'Individual Business'
                                 
                                 })

In [82]:
#cleaning up data
main = main[main['Hospital overall rating'] != "Not Available"]

In [85]:
# removing the dollars sign and turn these two columns to float datatype
main[' Average Total Payments '] = main[' Average Total Payments '].str.split('$').apply(lambda x: x[-1])

main['Average Medicare Payments'] = main['Average Medicare Payments'].str.split('$').apply(lambda x: x[-1])

main[' Average Total Payments '] = pd.to_numeric(main[' Average Total Payments '])
main['Average Medicare Payments'] = pd.to_numeric(main['Average Medicare Payments'])
main['Hospital overall rating'] = pd.to_numeric(main['Hospital overall rating'])
# df.dtypes

In [86]:
#Adding a new column "cost" this is the amount that patient has to pay
main['Cost'] = main[' Average Total Payments '] - main['Average Medicare Payments']

In [87]:
#clean up and merging with data4
main['Provider Id'] = main['Provider Id'] .astype(np.int64)

main = pd.merge(main,data4, on = 'Provider Id', how = 'outer')

In [88]:
#clean up
main= main.drop_duplicates()
main = main.dropna(how = "any") 

In [89]:
main.isnull().sum() # no missing values

Provider Id                  0
Provider Name                0
Provider Street Address      0
Provider City                0
Provider State               0
Provider Zip Code            0
 Average Total Payments      0
Average Medicare Payments    0
Hospital Type                0
Hospital Ownership           0
Hospital overall rating      0
BEDS                         0
Cost                         0
Latitude                     0
Longitude                    0
dtype: int64

In [90]:
main.shape

(122231, 15)

In [91]:
main.dtypes

Provider Id                    int64
Provider Name                 object
Provider Street Address       object
Provider City                 object
Provider State                object
Provider Zip Code            float64
 Average Total Payments      float64
Average Medicare Payments    float64
Hospital Type                 object
Hospital Ownership            object
Hospital overall rating      float64
BEDS                         float64
Cost                         float64
Latitude                     float64
Longitude                    float64
dtype: object

In [92]:
#export the main data 
main.to_csv("main_data.csv")

In [93]:
pwd

'C:\\Users\\SSF\\Desktop\\Group Project 1\\awesome_project\\MAIN DATA\\recipe of main_data'